In [ ]:
import numpy as np
import matplotlib.pyplot as plt



In [38]:
def design_matrix(x, degree):
    """Create design matrix for polynomial degree n with dimension determined
    by the size of input arrays and degree.

    Args:
        x (np.ndarray): x-values, 1D or 2D array
        y (np.ndarray): y-values, 1D or 2D array
        degree (int): order of polynomial degree
        
    Returns:
        np.ndarray: array with shape (len(x)*len(y), degree)
    """
    p = degree

    if len(x) == 1:
        x = x[0]
        print(x.shape)
        X = np.column_stack((np.ones_like(x), x))
        for i in range(2, p+1):
            X = np.column_stack((X, x**i))
        return X
    
    elif len(x) == 2:
        x, y = x[0], x[1]
        if len(x[0].shape) > 1:
            x = np.ravel(x)
            y = np.ravel(y)

        N = len(x)
        l = int((p + 1) * (p + 2) / 2)

        X = np.zeros((N, l))
        X[:, 0] = np.ones(N)

        for i in range(1, p + 1):
            q = int((i) * (i + 1) / 2)
            for k in range(i + 1):
                X[:, q + k] = (x**(i - k)) * (y**k)
        return X

    else:
        raise ValueError(f"{len(x)} is not a valid input dimension, must be 1D or 2D.")

In [ ]:
x1 = np.random.rand(10)
x2 = np.random.rand(10)
y = 2.0 + 5*x1**2 + x1*x2 + 0.1*np.random.randn(10)
degrees = np.arange(1, 5)
lmbda = 0.001

mse_history = []
r2_history = []

for p in degrees:
    X = design_matrix([x1, x2, x2], p)
    print(f"p = {p}:")
    print(X)